# **LOAD FILE SUARA**

Import libary dan data set dibutuhkan

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from glob import glob
import numpy as np
audio_files = np.array(glob('/content/drive/MyDrive/Pengenalan pola/Respiratory_Sound_Database/Respiratory_Sound_Database/*/*.wav'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ambil data dari csv dan melakukan perubahan dengan memberikan header

In [ ]:
import pandas as pd
path = "/content/drive/MyDrive/Pengenalan pola/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv"
data_pasien = pd.read_csv(path,dtype=str)
data_pasien.columns = ['ID', 'Diagnosis']

new_row = {'ID': '101', 'Diagnosis': 'URTI'}

data_pasien = data_pasien.append(new_row, ignore_index=True)
print(data_pasien)

      ID  Diagnosis
0    102    Healthy
1    103     Asthma
2    104       COPD
3    105       URTI
4    106       COPD
..   ...        ...
121  223       COPD
122  224    Healthy
123  225    Healthy
124  226  Pneumonia
125  101       URTI

[126 rows x 2 columns]


<ipython-input-41-57a735a24e14>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pasien = data_pasien.append(new_row, ignore_index=True)


### Ekstrasi suara dengan MFCC

In [ ]:
import librosa

mfcc = []
kelas = []
id = []
def MFCC(i):
    signal, sample_rate = librosa.load(i, sr=22050)

    numhop_length = 512
    num_fft = 2048
    MFCCs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_fft=num_fft, hop_length=numhop_length, n_mfcc=15)

    avr_mfcc = np.mean(MFCCs.T, axis=0)
    return avr_mfcc
def ID(i):
  ID_pasien = i[113:116]
  print(ID_pasien)
  print(i)
  diagnosis = data_pasien.loc[(data_pasien['ID'] == ID_pasien)].iloc[0]['Diagnosis']
  return diagnosis, ID_pasien

for i in audio_files:
  mfcc.append(MFCC(i))
  diagnosis, ID_pasien = ID(i)
  kelas.append(diagnosis)
  id.append(ID_pasien)

160
/content/drive/MyDrive/Pengenalan pola/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/160_1b3_Ar_mc_AKGC417L.wav
160
/content/drive/MyDrive/Pengenalan pola/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/160_1b4_Al_mc_AKGC417L.wav
160
/content/drive/MyDrive/Pengenalan pola/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/160_1b4_Ar_mc_AKGC417L.wav
160
/content/drive/MyDrive/Pengenalan pola/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/160_1b3_Pl_mc_AKGC417L.wav
160
/content/drive/MyDrive/Pengenalan pola/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/160_1b3_Pr_mc_AKGC417L.wav
160
/content/drive/MyDrive/Pengenalan pola/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/160_2b4_Pr_mc_AKGC417L.wav
160
/content/drive/MyDrive/Pengenalan pola/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/160_2b3_Lr_mc_AKGC4

Menambahkan data mfcc ke dataset

In [ ]:
labeled = {'ID': id,'mfcc': mfcc, 'kelas' : kelas}
labeled = pd.DataFrame(labeled)
print(labeled)

      ID                                               mfcc    kelas
0    160  [-353.52734, 105.35553, 66.756714, 36.95184, 2...     COPD
1    160  [-355.395, 112.53616, 64.79084, 33.579277, 25....     COPD
2    160  [-360.86325, 102.220245, 67.593575, 39.784485,...     COPD
3    160  [-388.23462, 104.58479, 50.404076, 44.28229, 3...     COPD
4    160  [-376.4118, 111.50186, 55.455875, 48.38389, 31...     COPD
..   ...                                                ...      ...
915  160  [-366.3713, 89.38993, 70.56693, 52.16571, 37.3...     COPD
916  159  [-451.59354, 92.46567, 56.998722, 36.22977, 31...  Healthy
917  160  [-352.94208, 111.87568, 61.937122, 30.847973, ...     COPD
918  160  [-350.29517, 97.178185, 77.04909, 56.454502, 3...     COPD
919  160  [-277.74405, 147.34485, 55.125916, 24.791193, ...     COPD

[920 rows x 3 columns]


menghapus data set yang tidak diperlukan

In [ ]:
labeled_filtered = labeled.drop(labeled[labeled["kelas"]=='LRTI'].index)
labeled_filtered = labeled_filtered.drop(labeled[labeled["kelas"]=='Bronchiectasis'].index)
labeled_filtered = labeled_filtered.drop(labeled[labeled["kelas"]=='Bronchiolitis'].index)
labeled_filtered = labeled_filtered.drop(labeled[labeled["kelas"]=='Asthma'].index)
print(labeled_filtered)

      ID                                               mfcc    kelas
0    160  [-353.52734, 105.35553, 66.756714, 36.95184, 2...     COPD
1    160  [-355.395, 112.53616, 64.79084, 33.579277, 25....     COPD
2    160  [-360.86325, 102.220245, 67.593575, 39.784485,...     COPD
3    160  [-388.23462, 104.58479, 50.404076, 44.28229, 3...     COPD
4    160  [-376.4118, 111.50186, 55.455875, 48.38389, 31...     COPD
..   ...                                                ...      ...
915  160  [-366.3713, 89.38993, 70.56693, 52.16571, 37.3...     COPD
916  159  [-451.59354, 92.46567, 56.998722, 36.22977, 31...  Healthy
917  160  [-352.94208, 111.87568, 61.937122, 30.847973, ...     COPD
918  160  [-350.29517, 97.178185, 77.04909, 56.454502, 3...     COPD
919  160  [-277.74405, 147.34485, 55.125916, 24.791193, ...     COPD

[888 rows x 3 columns]


mengubah label huruf menjadi label angka

In [ ]:
angka_kelas = {"kelas":{"COPD": 1, "URTI": 2,"Healthy": 3, "Pneumonia":4}}
labeled_filtered_encoded= labeled_filtered.replace(angka_kelas)
print(labeled_filtered_encoded)

      ID                                               mfcc  kelas
0    160  [-353.52734, 105.35553, 66.756714, 36.95184, 2...      1
1    160  [-355.395, 112.53616, 64.79084, 33.579277, 25....      1
2    160  [-360.86325, 102.220245, 67.593575, 39.784485,...      1
3    160  [-388.23462, 104.58479, 50.404076, 44.28229, 3...      1
4    160  [-376.4118, 111.50186, 55.455875, 48.38389, 31...      1
..   ...                                                ...    ...
915  160  [-366.3713, 89.38993, 70.56693, 52.16571, 37.3...      1
916  159  [-451.59354, 92.46567, 56.998722, 36.22977, 31...      3
917  160  [-352.94208, 111.87568, 61.937122, 30.847973, ...      1
918  160  [-350.29517, 97.178185, 77.04909, 56.454502, 3...      1
919  160  [-277.74405, 147.34485, 55.125916, 24.791193, ...      1

[888 rows x 3 columns]


memisahkan data label (y) dan data hasil mfcc (x)

In [ ]:
x=np.array(labeled_filtered_encoded['mfcc'].tolist())
y=np.array(labeled_filtered_encoded['kelas'].tolist())

In [ ]:
labeled_filtered.kelas.value_counts()

COPD         793
Pneumonia     37
Healthy       35
URTI          23
Name: kelas, dtype: int64

Memisahkan data set menjadi data training (80%) dan data validasi (20%)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, random_state=1)

In [ ]:
print('x train', x_train)
print('y train',y_train)

x train [[-394.13098     132.06749      43.721565   ...    3.4365747
     5.5193944     1.992099  ]
 [-442.6284       89.242004     70.94392    ...   19.180147
    16.0234       11.66346   ]
 [-424.29263      59.95856      50.33198    ...   16.5993
    14.197734     11.228347  ]
 ...
 [-271.97693     110.099045     48.37511    ...    1.3829559
     4.125616      0.72832274]
 [-263.28415     105.60273      47.72924    ...    3.0492077
     6.8028893     3.5890718 ]
 [-257.514       120.06972      54.683395   ...    3.3073585
     5.011369      2.5535133 ]]
y train [1 1 1 1 3 1 1 1 2 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3
 1 1 1 2 2 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 3 1 1
 1 1 1 1 1 3 1 3 2 1 1 1 1 1 1 1 1 1 1 3 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1
 3 1 1 1 1 1 4 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 4 3 1 1 1 1 1
 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 

# Catboost klasifikasi

In [ ]:
#sklearn matrices
from sklearn import metrics
#time
from time import*

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.2 MB/s eta 0:00:00


melakukan klasifikasi

In [ ]:
start = time()
import catboost as ctb
model_CBC = ctb.CatBoostClassifier()
model_CBC.fit(x_train, y_train)
print(model_CBC)
end = time()

Learning rate set to 0.077757
0:	learn: 1.2602721	total: 110ms	remaining: 1m 49s
1:	learn: 1.1629009	total: 127ms	remaining: 1m 3s
2:	learn: 1.0696830	total: 142ms	remaining: 47.3s
3:	learn: 1.0018025	total: 159ms	remaining: 39.6s
4:	learn: 0.9304575	total: 185ms	remaining: 36.9s
5:	learn: 0.8743482	total: 205ms	remaining: 33.9s
6:	learn: 0.8206416	total: 226ms	remaining: 32s
7:	learn: 0.7698647	total: 242ms	remaining: 30s
8:	learn: 0.7320890	total: 268ms	remaining: 29.6s
9:	learn: 0.6957641	total: 285ms	remaining: 28.2s
10:	learn: 0.6587549	total: 306ms	remaining: 27.5s
11:	learn: 0.6304527	total: 332ms	remaining: 27.3s
12:	learn: 0.6040562	total: 348ms	remaining: 26.4s
13:	learn: 0.5790685	total: 362ms	remaining: 25.5s
14:	learn: 0.5603107	total: 376ms	remaining: 24.7s
15:	learn: 0.5416639	total: 391ms	remaining: 24s
16:	learn: 0.5237933	total: 411ms	remaining: 23.7s
17:	learn: 0.5055488	total: 425ms	remaining: 23.2s
18:	learn: 0.4881788	total: 439ms	remaining: 22.7s
19:	learn: 0.472

melakukan evaluasi model

In [ ]:
predicted_y = model_CBC.predict(x_test)

In [ ]:
print(metrics.classification_report(y_test, predicted_y))
print(metrics.confusion_matrix(y_test, predicted_y))

              precision    recall  f1-score   support

           1       0.96      0.99      0.98       163
           2       0.00      0.00      0.00         0
           3       0.60      0.38      0.46         8
           4       0.67      0.29      0.40         7

    accuracy                           0.94       178
   macro avg       0.56      0.41      0.46       178
weighted avg       0.93      0.94      0.93       178

[[162   0   0   1]
 [  0   0   0   0]
 [  4   1   3   0]
 [  3   0   2   2]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = metrics.accuracy_score(y_test, predicted_y)
print()
print('prediction: ', predicted_y.reshape(1,-1))
print('actual: ', y_test)
print('accuracy: ', accuracy)


prediction:  [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1
  1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 3 1 2 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 3 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1]]
actual:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1
 1 3 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 3 1 1 1 1 3
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 3 1 3 1 1 1 1 1 1 4 1 1
 1 1 1 1 1 4 1 1 3 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1]
accuracy:  0.9382022471910112


In [ ]:
print("Waktu mulai :", start)
print("Waktu selesai :", end)
print("Durasi eksekusi program:", end-start)

Waktu mulai : 1700803817.4298496
Waktu selesai : 1700803837.8271885
Durasi eksekusi program: 20.3973388671875


In [ ]:
!pip install xgboost

# Decision Tree

In [ ]:
start = time()
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)
end = time()

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
print(metrics.classification_report(y_test, predicted_y))
print(metrics.confusion_matrix(y_test, predicted_y))

              precision    recall  f1-score   support

           1       0.96      0.99      0.98       163
           2       0.00      0.00      0.00         0
           3       0.60      0.38      0.46         8
           4       0.67      0.29      0.40         7

    accuracy                           0.94       178
   macro avg       0.56      0.41      0.46       178
weighted avg       0.93      0.94      0.93       178

[[162   0   0   1]
 [  0   0   0   0]
 [  4   1   3   0]
 [  3   0   2   2]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
accuracy = metrics.accuracy_score(y_test, y_pred)
print('prediction: ', y_pred)
print('actual: ', y_test)
print('accuracy: ', accuracy)

prediction:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1
 4 4 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 3
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 3 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 3 1 1 3 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1]
actual:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1
 1 3 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 3 1 1 1 1 3
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 3 1 3 1 1 1 1 1 1 4 1 1
 1 1 1 1 1 4 1 1 3 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1]
accuracy:  0.9101123595505618


In [ ]:
print("Waktu mulai :", start)
print("Waktu selesai :", end)
print("Durasi eksekusi program:", end-start)

Waktu mulai : 1700804117.949222
Waktu selesai : 1700804118.6373842
Durasi eksekusi program: 0.688162088394165


# Keras Klasifikasi

In [ ]:
start = time()
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(12, activation='relu'),
#    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.03),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(x_train, y_train, epochs=10,verbose=2)
model.summary()
end = time()

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


23/23 - 3s - loss: 0.0000e+00 - accuracy: 0.0521 - precision: 1.0000 - recall: 0.0535 - 3s/epoch - 130ms/step
Epoch 2/10
23/23 - 0s - loss: 0.0000e+00 - accuracy: 0.7648 - precision: 1.0000 - recall: 0.8662 - 101ms/epoch - 4ms/step
Epoch 3/10
23/23 - 0s - loss: 0.0000e+00 - accuracy: 0.8859 - precision: 1.0000 - recall: 0.9986 - 128ms/epoch - 6ms/step
Epoch 4/10
23/23 - 0s - loss: 0.0000e+00 - accuracy: 0.8859 - precision: 1.0000 - recall: 0.9986 - 72ms/epoch - 3ms/step
Epoch 5/10
23/23 - 0s - loss: 0.0000e+00 - accuracy: 0.8873 - precision: 1.0000 - recall: 1.0000 - 80ms/epoch - 3ms/step
Epoch 6/10
23/23 - 0s - loss: 0.0000e+00 - accuracy: 0.8873 - precision: 1.0000 - recall: 1.0000 - 99ms/epoch - 4ms/step
Epoch 7/10
23/23 - 0s - loss: 0.0000e+00 - accuracy: 0.8873 - precision: 1.0000 - recall: 1.0000 - 119ms/epoch - 5ms/step
Epoch 8/10
23/23 - 0s - loss: 0.0000e+00 - accuracy: 0.8873 - precision: 1.0000 - recall: 1.0000 - 107ms/epoch - 5ms/step
Epoch 9/10
23/23 - 0s - loss: 0.0000e+0

In [ ]:
prediction = model.predict(x_test)
accuracy = metrics.accuracy_score(y_test, prediction)
print()
print('prediction: ', prediction.reshape(1,-1))
print('actual: ', y_test)
print('accuracy: ', accuracy)

6/6 [==============================] - 0s 4ms/step

prediction:  [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
actual:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1
 1 3 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 3 1 1 1 1 3
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 3 1 3 1 1 1 1 1 1 4 1 1
 1 1 1 1 1 4 1 1 3 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1]
accuracy:  0

In [ ]:
print("Waktu mulai :", start)
print("Waktu selesai :", end)
print("Durasi eksekusi program:", end-start)

Waktu mulai : 1700804234.848965
Waktu selesai : 1700804253.8645778
Durasi eksekusi program: 19.015612840652466


In [ ]:
start = time()
from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(hidden_layer_sizes=(12), max_iter=10,activation = 'relu',solver='adam',random_state=1)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
end = time()

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(y_test, y_pred)
print()
print('prediction: ', y_pred.reshape(1,-1))
print('actual: ', y_test)
print('accuracy: ', accuracy)


prediction:  [[4 4 4 4 1 4 4 1 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4 4 2 4 4 2 4 4 4 4 4 4 4 4
  4 4 4 1 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 2 4 1 4 4 4 4 4 4 4 4 1 4 4
  2 2 2 4 1 2 4 4 1 4 4 4 4 4 4 4 4 2 4 4 4 2 4 4 4 1 4 4 4 4 4 4 1 4 4 4
  2 4 1 4 4 4 4 4 1 1 2 4 1 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 1 1 1 4 4 4 2 4
  4 4 4 2 4 4 4 4 4 2 2 4 4 4 4 4 4 4 4 2 4 2 4 4 4 4 4 4 4 4 4 4 4 4]]
actual:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1
 1 3 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 3 1 1 1 1 3
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 3 1 3 1 1 1 1 1 1 4 1 1
 1 1 1 1 1 4 1 1 3 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1]
accuracy:  0.11235955056179775


In [ ]:
print("Waktu mulai :", start)
print("Waktu selesai :", end)
print("Durasi eksekusi program:", end-start)

Waktu mulai : 1700804330.345645
Waktu selesai : 1700804330.423328
Durasi eksekusi program: 0.0776829719543457
